In [ ]:
!pip install medpy

In [ ]:
from google.colab import auth, drive
from googleapiclient.discovery import build

auth.authenticate_user()
drive.mount('/content/drive')

In [ ]:
from PIL import Image
import os
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt, cm

from medpy.metric import dc, jc
from medpy.io import load, header

from tqdm import notebook, trange

SEED = 42

In [ ]:
def path(fname, num):
    # return f'drive/My Drive/CHAOS/CHAOS_Train_Sets/Train_Sets/CT/' + fname
    return f'drive/My Drive/Anon_Liver/Anon_Liver/Abdomen - 3928/' + fname

def get_data(num):
    X_path = path('', num)
    X_filenames = []

    for directory in notebook.tqdm(os.listdir(X_path)[:-1], position=0):
        for dirName, subdirList, fileList in os.walk(X_path+directory):
            for filename in fileList:
                if ".dcm" in filename.lower():
                    X_filenames.append(os.path.join(dirName, filename))

    return sorted(X_filenames)

In [ ]:
X_filenames1 = get_data(1)
X_filenames = X_filenames1[:100]


print(X_filenames)

In [ ]:
def buffer_imgs(filenames, folder='buffer'):
    files = []
    if not os.path.exists(folder):
        os.makedirs(folder)
    for filename in notebook.tqdm(filenames, position=0):
        img, header = load(filename)
        pil = Image.fromarray(img.squeeze())
        fname = folder + '/' + filename.replace('/', '-')
        fname = fname + '.tiff'
        pil.save(fname, 'TIFF', compression='none')
        files.append(fname)
    return pd.DataFrame(files)

In [ ]:
X = buffer_imgs(X_filenames)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model


val_gen_params = {
    'x_col': 0,
    'target_size': (512, 512),
    'color_mode': 'grayscale',
    'batch_size': 1,
    'class_mode': None,
    'shuffle': False,
    'seed': SEED,
}

idg_test_data = ImageDataGenerator(
    samplewise_center=True,
    samplewise_std_normalization=True
                                  )
idg_test_mask = ImageDataGenerator(rescale=1./255)

model = load_model('unet_r(2).h5')


In [ ]:
def evaluate(x_names, y_names, set_name='evaluating', plot_pairs=0):
    val_gen_params['batch_size'] = 1
    print(set_name)
    dices = []
    jccrs = []
    mdices = []
    mjccrs = []
    data_g = idg_test_data.flow_from_dataframe(x_names, **val_gen_params)
    mask_g = idg_test_mask.flow_from_dataframe(y_names, **val_gen_params)
    for i, image_mask in enumerate(zip(notebook.tqdm(data_g), mask_g)):
        if i > x_names.shape[0] // val_gen_params['batch_size']:
            break
        image, mask = image_mask
        if mask.max() == 0:
            continue
        p = model.predict(image).astype('uint8')
        dice = dc(p, mask)
        dices.append(dice)
        try:
            jccr = jc(p, mask)
        except ZeroDivisionError as e:
            jccr = 1
        jccrs.append(jccr)

        if plot_pairs:
            plot_pairs -= 1
            fig, (ax1, ax2) = plt.subplots(1,2, figsize=(5,10))
            ax1.imshow(image.squeeze())
            ax1.imshow(mask.squeeze(), alpha=0.5, cmap='autumn')
            ax1.set_title('ground')
            ax2.imshow(image.squeeze())
            ax2.imshow(p.squeeze(), alpha=0.5, cmap='autumn')
            ax2.set_title('predicted, dc={0:.2f}'.format(dice))
            plt.show()

    print('dice:', np.mean(dices))
    print('jccr:', np.mean(jccrs))
    print('-------------')
    return jccr